In [1]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWhirb3zsx7fBcoQDCM4c56KfnQuIXvSOJd9dDsXk6k450JuuVfNA6Y

Successfully saved authorization token.


In [2]:
import os
import json
import earthpy as et
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import IPython.display as disp
%matplotlib inline

# change the global options that Geopandas inherits from
pd.set_option('display.max_columns',None)

In [3]:
# Change directory to landslide-detect data path
data_path = os.path.join(et.io.HOME, "earth-analytics", "landslide-detect")
if os.path.exists(data_path):
    os.chdir(data_path)
else:
    os.makedirs(data_path)
    print('The new directory is created!')
    os.chdir(data_path)

print('Current working directory is set to: ', os.getcwd())

Current working directory is set to:  /Users/leahmanak/earth-analytics/landslide-detect


In [13]:
def create_dataframe(data_file):
    """
    Reads a csv file into a GeoPandas DataFrame. Sets the index to the 'slide.id'
    and sorts the geoDataFrame by the new index. Converts the date+time column to
    a DateTime object and drops the time. Converts the reported geocoordinates to
    a geoPandas geometry point.

    Parameters
    ----------
    data_file : str

    Returns
    -------
    dataframe
        sorted by index
    """

    gdf = gpd.read_file(data_file, index_col='slide.id').sort_values(by='slide.id')
    gdf = gdf.set_index('slide.id')
    # Convert to DatTime object, drop time
    gdf['slide.date'] = pd.to_datetime(gdf['slide.date'])
    gdf['slide.date'] = gdf['slide.date'].dt.date
    # Convert reported geocoordinates to geometery
    gdf[['lon', 'lat']] = gdf[['lon', 'lat']].astype(float)
    geometry = [Point(xy) for xy in zip(gdf.lon, gdf.lat)]
#     gdf = gdf.drop(['lon', 'lat'], axis=1)
    gdf = GeoDataFrame(gdf, crs="EPSG:4326", geometry=geometry)

    return gdf
landslide_gdf = create_dataframe('landslides.verified.csv')
landslide_gdf.head()

,slide.date,location,type,trigger,size,lon,lat,location_accuracy,event_title,admin_division_name,ge.lat,ge.lon,is.exact,slide.index,geometry
slide.id,,,,,,,,,,,,,,,
10032,2017-07-11,Cassville rock slide,landslide,rain,medium,-90.999234,42.721873,5km,Cassville rock slide,NA,NA,NA,FALSE,115,POINT (-90.99923 42.72187)
10034,2016-09-20,Balligomingo Road Mudslide,mudslide,rain,medium,-75.325703,40.070575,1km,Balligomingo Road Mudslide,NA,40.070575000000005,-75.32570277777778,TRUE,127,POINT (-75.32570 40.07058)
10063,2016-07-02,Mudslide closes Highway 837,mudslide,rain,medium,-112.911742,51.575712,1km,Mudslide closes Highway 837,NA,NA,NA,FALSE,152,POINT (-112.91174 51.57571)
10120,2017-01-06,Multiple Rockslides along Highway 1 near Ragge...,rock_fall,rain,medium,-121.328879,35.779001,1km,Multiple Rockslides along Highway 1 near Ragge...,NA,NA,NA,FALSE,166,POINT (-121.32888 35.77900)
10121,2017-03-05,New Camaldoli Hermitage Monks Trapped by Mudsl...,mudslide,rain,medium,-121.531978,36.018678,5km,New Camaldoli Hermitage Monks Trapped by Mudsl...,NA,36.018677777777775,-121.53197777777778,TRUE,162,POINT (-121.53198 36.01868)


In [5]:
def image_search(gdf):
    """
    Iterates through a geoDataFrame and searches Google Earth Engine
    based on certain parameters within the geoDataFrame. Buffers central
    geometery search point by 1000 meters, event date by 180 days reciprocal. 

    
    Parameters
    ----------
    data_file : geodataframe

    Returns
    -------
    dataframe
        appended with imagery timestamplist
    
    """

    # Set empty list
    results = []
    

    BASE_DATE = ee.Date(landslide_gdf['slide.date'])

    # Data search in Google Earth Engine
    im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
               .filterBounds(gdf['geometry'])
               .filter(ee.Geometry.Point.buffer({'distance': 1000}))
               .filterDate(BASE_DATE, 
                           BASE_DATE.advance(-180, 'days'), 
                           BASE_DATE.advance(180, 'days'))
#                .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
               .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
               .sort('date'))
    timestamplist = (im_coll.aggregate_array('date')
                            .map(lambda d: ee.String('T').cat(ee.String(d)))
                            .getInfo())
    
    # Add new column to geopandas DataFrame for avaiable data
    for value in timestamplist:
        if value == True:
            results.append(timestamplist)
        else:
            results.append('No imagery available')

    return gdf

In [6]:
# for id, location, point in zip(landslide_df.index,
#                                landslide_df['location'],
#                                landslide_df['point'],
#                                ):
#     print(id, location, point)

In [7]:
# for row in gdf.iterrows():
#     print(gdf['geometry'])

In [8]:
# for row in landslide_gdf.iterrows():
#     print(landslide_gdf['geometry'])

In [9]:
# landslide_gdf = create_dataframe('landslides.verified.copy.csv')
# landslide_gdf.head()

In [10]:
image_search(landslide_gdf)

EEException: Invalid argument specified for ee.Date(): slide.id
10032    2017-07-11
10034    2016-09-20
10063    2016-07-02
10120    2017-01-06
10121    2017-03-05
            ...    
9662     2016-10-21
9734     2017-05-20
9806     2017-05-19
9844     2017-06-19
9967     2017-07-01
Name: slide.date, Length: 228, dtype: object